In [1]:
Fixed_Length = 256

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# BARTpho-syllable
# syllable_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
# bartpho_syllable = AutoModel.from_pretrained("vinai/bartpho-syllable")
# TXT = "Chúng tôi là những nghiên cứu viên."
# input_ids = syllable_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_syllable(input_ids)

# BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word-base")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word-base")
# to cuda
bartpho_word.to("cuda")
bartpho_word.eval()
for param in bartpho_word.parameters():
    param.requires_grad = False
# TXT = "Chúng_tôi là những nghiên_cứu_viên ."
# input_ids = word_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_word(input_ids.to("cuda"))

# input_ids = word_tokenizer.encode(TXT, return_tensors="pt").to("cuda")
# print(input_ids)
# # check if input_ids is in cuda
# features = bartpho_word(input_ids).last_hidden_state[0]

d:\AIChallenge\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\AIChallenge\venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# Word segmentation
import py_vncorenlp

# py_vncorenlp.download_model(save_dir=r"D:\VQA\Notebook\VnCoreNLP")

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"],
    save_dir=r"D:\AIChallenge\VnCoreNLP",
)

# change dir to C:\Users\Asus.LAPTOP-8EU9PHJL.000\Desktop\VQA\Notebook
import os

os.chdir(r"D:\AIChallenge")


def Word_Segmentation(text):
    output = rdrsegmenter.word_segment(text)
    if len(output) > 0:
        # join all words in the list
        res = ""
        for i in output:
            res += i + " "
        return res
    else:
        output = ""
    return output


print(Word_Segmentation("Chúng tôi là những nghiên cứu viên."))

Chúng_tôi là những nghiên_cứu_viên . 


In [4]:
import os


def scan_file_path_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            lst.append(os.path.join(root, file))
    return lst


def scan_file_name_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            # remove the extension
            file_name = os.path.splitext(file)[0]
            lst.append(file_name)
    return lst


audio_text = r"Data\rawData\Audio_text"

In [5]:
path_lst = scan_file_path_in_folder("Data\\PreporcessData\\Audio_key_text\\L01_V001")
name_lst = scan_file_name_in_folder(audio_text)
print(path_lst)
print(name_lst)

['Data\\PreporcessData\\Audio_key_text\\L01_V001\\001.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\002.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\003.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\004.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\005.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\006.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\007.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\008.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\009.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\010.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\011.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\012.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\013.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\014.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\015.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\016.txt', 'Data\\PreporcessData\\Audio_key_text\\L01_V001\\017.tx

In [6]:
# path_lst = scan_file_path_in_folder(audio_text)
# name_lst = scan_file_name_in_folder(audio_text)

In [7]:
# path = "Data\\PreporcessData\\keyframes_ImageCaption"
# path_lst = scan_file_path_in_folder(path)
# name_lst = scan_file_name_in_folder(path)
# print(path_lst)
# print(name_lst)

In [8]:
# for path, name in tqdm(zip(path_lst, name_lst)):
#     out = f"Data\PreporcessData\keyframes_ImageCaption_bartpho\{name}.pt"
#     # check if the file is already processed
#     if os.path.exists(out):
#         continue
#     # read text
#     with open(path, "r", encoding="utf-8") as f:
#         text = f.read()
#     # join text
#     text = Word_Segmentation(text)
#     # remove every '.', ',', '!', '?'
#     text = text.replace(".", " ").replace(",", " ").replace("!", " ").replace("?", " ")
#     # clean ' 'ArithmeticError
#     text = " ".join(text.split())
#     text = text.split(" ")
#     num_chunks = (len(text) + Fixed_Length - 1) // Fixed_Length
#     max_len_each_chunk = (len(text) + num_chunks - 1) // num_chunks
#     lst_text = []
#     for i in range(0, len(text), max_len_each_chunk):
#         lst_text.append([" ".join(text[i : i + max_len_each_chunk])])
#     lst_ids = [
#         word_tokenizer.encode(
#             i[0],
#             return_tensors="pt",
#             padding="max_length",
#             max_length=max_len_each_chunk,
#             truncation=True,
#             add_special_tokens=False,
#         ).to("cuda")
#         for i in lst_text
#     ]
#     with torch.no_grad():
#         # check if input_ids is in cuda
#         lst_outputs = [bartpho_word(i) for i in lst_ids]
#         lst_features = [i.last_hidden_state[0] for i in lst_outputs]
#         # stack all features to a tensor
#         features = torch.stack(lst_features)
#     # save features
#     torch.save(features, out)

In [9]:
for name in tqdm(name_lst):
    path_lst = scan_file_path_in_folder(f"Data\\PreporcessData\\Audio_key_text\\{name}")
    sub_name_lst = scan_file_name_in_folder(
        f"Data\\PreporcessData\\Audio_key_text\\{name}"
    )
    for path, sub_name in zip(path_lst, sub_name_lst):
        out_path = f"Data\\PreporcessData\\Audio_key_text_bartpho\\{name}_{sub_name}.pt"
        # check if the file is already exist
        if os.path.exists(out_path):
            continue
        # read text
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
        # join text
        text = Word_Segmentation(text)
        # remove every '.', ',', '!', '?'
        text = (
            text.replace(".", " ").replace(",", " ").replace("!", " ").replace("?", " ")
        )
        # clean ' 'ArithmeticError
        text = " ".join(text.split())
        # split the text into multiple parts with at most 40 words
        text = text.split(" ")
        num_chunks = (len(text) + Fixed_Length - 1) // Fixed_Length
        max_len_each_chunk = (len(text) + num_chunks - 1) // num_chunks
        # if (len(text) < 5):
        #     continue
        lst_text = []
        for i in range(0, len(text), max_len_each_chunk):
            lst_text.append([" ".join(text[i : i + max_len_each_chunk])])
        lst_ids = [
            word_tokenizer.encode(
                i[0],
                return_tensors="pt",
                padding="max_length",
                max_length=max_len_each_chunk,
                truncation=True,
                add_special_tokens=False,
            ).to("cuda")
            for i in lst_text
        ]
        with torch.no_grad():
            # check if input_ids is in cuda
            lst_outputs = [bartpho_word(i) for i in lst_ids]
            lst_features = [i.last_hidden_state[0] for i in lst_outputs]
            # stack all features to a tensor
            features = torch.stack(lst_features)
        # save features
        torch.save(features, out_path)

100%|██████████| 363/363 [21:38<00:00,  3.58s/it]


In [10]:

# for path, name in tqdm(zip(path_lst, name_lst)):
#     # read text
#     with open(path, "r", encoding="utf-8") as f:
#         text = f.read()
#     # join text
#     text = Word_Segmentation(text)
#     # remove every '.', ',', '!', '?'
#     text = text.replace(".", " ").replace(",", " ").replace("!", " ").replace("?", " ")
#     # clean ' 'ArithmeticError
#     text = " ".join(text.split())
#     # write text
#     with open(
#         f"Data\PreporcessData\Audio_text_wseg\{name}.txt", "w", encoding="utf-8"
#     ) as f:
#         # write to each line
#         f.write(text)
#     # split the text into multiple parts with at most 40 words
#     text = text.split(" ")
#     lst_text = []
#     for i in range(0, len(text), 40):
#         lst_text.append([" ".join(text[i : i + 40])])
#     lst_ids = [
#         word_tokenizer.encode(
#             i[0],
#             return_tensors="pt",
#             padding="max_length",
#             max_length=50,
#             truncation=True,
#         ).to("cuda")
#         for i in lst_text
#     ]
#     with torch.no_grad():
#         # check if input_ids is in cuda
#         lst_outputs = [bartpho_word(i) for i in lst_ids]
#         lst_features = [i.last_hidden_state[0] for i in lst_outputs]
#         # stack all features to a tensor
#         features = torch.stack(lst_features)
#     # save features
#     torch.save(features, f"Data\PreporcessData\Audio_text_bartpho\{name}.pt")